In [16]:
import os
import pandas as pd
import subliminal
from babelfish import Language
from subliminal import Video, Movie
from tqdm import tqdm
from glob import glob

In [2]:
df = pd.read_csv('dataset/tmdb_5000_movies.csv')
# filter only US movies
df = df[df.production_countries.str.contains("United States of America")]
df = df.sort_values(by='release_date', ascending=False)
df = df.reset_index(drop=True)

In [3]:
os.makedirs("dataset/subtitles", exist_ok=True)

In [ ]:
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    if len(glob("dataset/subtitles/{}.*".format(row.title))) == 0:  # file doesn't exist yet
        #print(row.id, row.title)
        video = Video.fromname('{}.mkv'.format(row.title))
        # print(video)
        if not isinstance(video, subliminal.video.Movie):
            print("Error: ", video, row.id)
            continue

        best_subtitles = subliminal.download_best_subtitles(
            [video], {Language('eng')})
        best_subtitle = best_subtitles[video]
        if not best_subtitles[video]:  # if no subtitles are found
            print("Error: ", video, row.id)
            continue

        best_subtitle = best_subtitle[0]
        subliminal.save_subtitles(
            video, [best_subtitle], directory="dataset/subtitles")
        # print()